In [6]:
import os
import shutil
import zipfile
import pandas as pd

In [7]:
# output directory to store the results
outDir = "/Users/duanyueyun/Desktop/Capstone/Decompiled_files"
# input directory that contains the APK files
inDir = "/Users/duanyueyun/Desktop/Capstone/APK_files"
# directory to d2j-dex2jar.sh
dex2jar = "/Users/duanyueyun/Desktop/Capstone/dex2jar-2.0/d2j-dex2jar.sh"
# directory to jd-cli.jar
jd_cli = "/Users/duanyueyun/Desktop/Capstone/jd-cli-1.1.0.Final-dist/jd-cli.jar"

In [8]:
# get all apk filenames
apks=os.listdir(inDir)

In [9]:
# just check all files have been downloaded, can skip
repack_pairs = pd.read_csv("~/Desktop/Capstone/Download apk/repackaging_pairs.txt", sep=',')
mine = repack_pairs.iloc[12001:,:]
sha256s = list(mine['SHA256_ORIGINAL'].unique())
sha256s += list(mine['SHA256_REPACKAGE'].unique())
downloaded = [apk.split('.')[0] for apk in apks]
for i in sha256s:
    if i not in downloaded:
        print(i)

In [14]:
for i in range(1):

    apk = apks[i]
    apk_name=apk.split('.')[0]
    
    # skip files with extension other than apk
    if apk.split('.')[1]!='apk':
        continue
    
    # create a path named by the app's SHA256
    output_folder = os.path.join(outDir, apk_name)
    
    # skip if the app has already been processed
    if os.path.exists(output_folder):
        continue
        
    # create the folder
    os.mkdir(output_folder)

    # path of the .jar file
    output_jar = os.path.join(output_folder, apk_name+'.jar')
    # path of the apk file
    in_apk = os.path.join(inDir, apk)
    # obtain the .jar file
    os.system("sh {} -f -o {} {}".format(dex2jar, output_jar, in_apk))
    
    # create a folder named java_sources to store the java files
    jar_folder = os.path.join(output_folder, "java_sources")
    os.mkdir(jar_folder)
    
    # obtain the java files
    os.system("java -jar {} -od {} {}".format(jd_cli, jar_folder, output_jar))
    
    # remove the .jar file
    os.remove(output_jar)
    
    # obtain xml and smali files
    other_folder = os.path.join(output_folder, "other")
    os.system('apktool d {} -o {}'.format(in_apk, other_folder))
    
    # delete files that we do not want
    for file in os.listdir(other_folder):
        if file == 'AndroidManifest.xml' or file == 'smali':
            continue
        path = os.path.join(other_folder, file)
        
        if os.path.isdir(path):
            shutil.rmtree(path)
        else:
            os.remove(path)